In [1]:
# LightGBM

# Installing LightGBM
# Open a terminal (Mac & Linux) or the anaconda prompt (Windows) and enter the following command:
# conda install -c conda-forge lightgbm

In [2]:
# Importing the libraries
import numpy as np
import pandas as pd


In [3]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values #Varibles independientes
y = dataset.iloc[:, 13].values   #Objetivo

In [4]:
dataset.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [5]:
dataset.shape

(10000, 14)

In [6]:
#Hemos conseguido datos de 10.000 clientes y hemos visto si han permanecido
#o no en el banco (Exit) durante 6 meses. Exited=1 si se fue del banco
#Exited=0 si se quedó en el banco

#Objetivo: Modelo de machine learning que sea capaz de predecir si se queda
#o se va. Vamos a buscar el mejor modelo.

In [7]:
#Variables categóricas: Geography (France, Spain Germany), Gender

In [8]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#Primero las etiqueta
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) #Geography
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) #Gender

#Aplica OneHotEncoder solo a Geography (1), ya que Gender como solo
#tiene 2 categorias con el LabelEncoder basta ya que lo ha etiquetado
#como 0 y 1 (con una sola columna nos vale para una columna con dos
#categorias)

onehotencoder = OneHotEncoder(categorical_features = [1]) #Solo la columna Geography
X = onehotencoder.fit_transform(X).toarray()

#Las nuevas columnas las añade al principio (Francia Alemania España)
                                            #   1       0       0
                                            #   0       0       1
                                            #   1       0       0
#Para evitar el problema de la multicolinialidad (trampa de las variables
#categóricas, siempre se quita una de las columnas. Si hay tres paises
#nos quedamos con dos columnas. En la siguiente instrucción elimino
#la primera de las columnas
            
X = X[:, 1:]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [9]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [10]:
# Splitting the dataset into the Training set and Test set (80%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [11]:
X_train.shape

(8000, 11)

In [12]:
X_test.shape

(2000, 11)

In [13]:
#Entrenamiento del modelo
# Fitting LightGBM to the Training set
import lightgbm as lgb


/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [14]:
#Se crea una estructura de entrenamiento especial que pide la librería 
#LigthGBM. Es una librería de Microsoft y esta es su estructura
training_data = lgb.Dataset(data = X_train, label = y_train)

#Configuramos parámetros del algoritmo. Tiene más de 100 hyperparámetros
#Se podría usar GreedSearch.....
#num_leaves: número de hojas para limitar el crecimiento horizontal y que 
#crezda en profundidad
#num_trees: número máximo de árboles
#objective: tipo de árbol, en nuestro caso es de clasificación binario

params = {'num_leaves': 31, 'num_trees': 100, 'objective': 'binary'}

#https://lightgbm.readthedocs.io/en/latest/Parameters.html

#Metricas de medidas a optimizar
#'auc': area bajo la curva
#binary_logloss: función de pérdidas clas
params['metric'] = ['auc', 'binary_logloss']

#Modelo
#num_boost_roud: número de vueltas en el bucle del Gradient Boosted Tree
classifier = lgb.train(params = params,
                       train_set = training_data,
                       num_boost_round = 10)

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [15]:
# Predicting the Test set results
#Al haber utilizado como función de pérdidas binary_logloss el resultado
#de la predicción es una probabilidad
prob_pred = classifier.predict(X_test)

In [16]:
prob_pred  #cliente 1: 39% que se vaya
           #cliente 2: 21% que se vaya.....

array([0.39400398, 0.21188121, 0.10716891, ..., 0.04995754, 0.18423094,
       0.24545014])

In [17]:

#Como el resultado es una predicción voy a asignar el valor 1 si la 
#probabilidad es >= 0.5 y 0 si la probabilidad es menor de 0.5.
#Se ha decidido usar el 50% pero podríamos poner otro límite
y_pred = np.zeros(len(prob_pred))
for i in range(0, len(prob_pred)):
    if prob_pred[i] >= 0.5:
        y_pred[i] = 1
    else:  
        y_pred[i] = 0

In [18]:
y_pred [:20]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0.])

In [19]:
#Comparamos los resultados obtenidos con la predicción con los valores
#reales, así también veremos si el 50% que hemos decidido es bueno
#Para esto se utiliza la matriz de confusión (se juntan en filas y columnas
#el valor predicho con el real)

In [20]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred) #filas valore reales
                                      #columnas predicciones del algoritmo

In [21]:
cm

array([[1499,   96],
       [ 188,  217]])

In [22]:
#Buenas clasificaciones 0,0: 1499 clientes que no se marchan
#                       1,1: 217 clientes que se marchan
#Malas clasificaciones  0,1: 96 el cliente no se marcha pero predecimos que si
#                       1,0: 188 el cliente se marcha y predecimos que no

In [23]:
#Medida de la precisión
# Getting the Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred, y_test) * 100
print("Accuracy: {:.0f} %".format(accuracy))

Accuracy: 86 %


In [24]:
#Usamos validación cruzada en k bloques (nfold=10 veces) y luego
#hacemos la media

In [25]:
# Applying k-Fold Cross Validation
params = {'num_leaves': 31, 'num_trees': 100, 'objective': 'binary'}
params['metric'] = ['auc']
cv_results = lgb.cv(params = params,
                    train_set = training_data,
                    num_boost_round = 10,
                    nfold = 10)
average_auc = np.mean(cv_results['auc-mean'])
print("Average AUC: {:.0f} %".format(accuracy))

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:502: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Average AUC: 86 %


In [26]:
########### LightGBM con GridSearch #########

In [27]:
from sklearn.model_selection import GridSearchCV

In [28]:
gridParams = { 
    'num_leaves': [10,20,31,50],
    'num_trees': [100,200,300],
    'objective': ['binary'],
    


}

In [29]:
grid = GridSearchCV (lgb.LGBMClassifier(),param_grid=gridParams, cv=5)

In [30]:
grid.fit(X_train,y_train)

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params.

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params.

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'num_leaves': [10, 20, 31, 50], 'num_trees': [100, 200, 300], 'objective': ['binary']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [31]:
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))

best mean cross-validation score: 0.863


In [32]:
print("best parameters: {}".format(grid.best_params_))

best parameters: {'num_leaves': 20, 'num_trees': 100, 'objective': 'binary'}


In [33]:
#Entreno con los mejores parámetros

In [34]:

num_leaves_optimo = grid.best_params_['num_leaves']
num_trees_optimo = grid.best_params_['num_trees']
objetive_optimo = grid.best_params_['objective']

In [35]:
params = {'num_leaves': num_leaves_optimo, 
          'num_trees_optimo': num_trees_optimo,
          'objective': objetive_optimo,
           }

params['metric'] = ['auc', 'binary_logloss']

#Modelo
#num_boost_roud: número de vueltas en el bucle del Gradient Boosted Tree
classifier = lgb.train(params = params,
                       train_set = training_data,
                       num_boost_round = 10)

In [36]:
prob_pred = classifier.predict(X_test)

In [37]:
#Como el resultado es una predicción voy a asignar el valor 1 si la 
#probabilidad es >= 0.5 y 0 si la probabilidad es menor de 0.5.
#Se ha decidido usar el 50% pero podríamos poner otro límite
y_pred = np.zeros(len(prob_pred))
for i in range(0, len(prob_pred)):
    if prob_pred[i] >= 0.5:
        y_pred[i] = 1
    else:  
        y_pred[i] = 0

In [38]:
#Medida de la precisión
# Getting the Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred, y_test) * 100
print("Accuracy: {:.0f} %".format(accuracy))

Accuracy: 87 %


In [39]:
# Applying k-Fold Cross Validation
params = {'num_leaves': num_leaves_optimo, 
          'num_trees_optimo': num_trees_optimo,
          'objective': objetive_optimo}
params['metric'] = ['auc']
cv_results = lgb.cv(params = params,
                    train_set = training_data,
                    num_boost_round = 10,
                    nfold = 10)
average_auc = np.mean(cv_results['auc-mean'])
print("Average AUC: {:.0f} %".format(accuracy))

Average AUC: 87 %
